## 1. Setup and Imports

In [1]:
import sys
import warnings
warnings.filterwarnings('ignore')

if 'google.colab' in sys.modules:
    src_path = '/content/Multimodal-Vehicle-Damage-Assessor/src'
    os.chdir('/content/Multimodal-Vehicle-Damage-Assessor')
    sys.path.insert(0, src_path)
else:
    sys.path.append('../src')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Import custom modules
from config import *
from data_loader import *
from visualization import *
from train_utils import *
from yolo_utils import *

ModuleNotFoundError: No module named 'ultralytics'

## 2. Data Exploration

In [ ]:
# Check class distribution
train_dist = get_class_distribution(TRAIN_DIR)
val_dist = get_class_distribution(VAL_DIR)

plot_class_distribution(train_dist, val_dist)

In [ ]:
visualize_sample_images(TRAIN_DIR, num_samples=4)
plt.show()

## 3. Prepare Data Generators for CNN Models

In [ ]:
# Create data generators with augmentation
train_generator, val_generator, class_indices = create_data_generators(
    img_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

## 4. Train Transfer Learning Models

We'll train three popular CNN architectures with ImageNet pre-trained weights.

### 4.1 ResNet50

In [ ]:
print("="*70)
print("Training ResNet50")
print("="*70)

# Build model
resnet_model = build_transfer_learning_model('resnet50', img_size=IMG_SIZE, num_classes=3)
print(f"\nModel architecture:")
resnet_model.summary()

# Train model
resnet_history = resnet_model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS,
    callbacks=get_callbacks('resnet50', patience=15),
    verbose=1
)

# Plot training history
plot_training_history(resnet_history, 'ResNet50')
plt.show()

print("\n✓ ResNet50 training completed!")

In [ ]:
# Evaluate ResNet50
print("Evaluating ResNet50 on validation set...")
resnet_results = evaluate_model(resnet_model, val_generator)

print(f"\nResNet50 Results:")
print(f"  Accuracy:  {resnet_results['accuracy']:.4f}")
print(f"  Precision: {resnet_results['precision']:.4f}")
print(f"  Recall:    {resnet_results['recall']:.4f}")
print(f"  F1-Score:  {resnet_results['f1_score']:.4f}")

# Save results
save_model_results('resnet50', resnet_results, resnet_history)

# Plot confusion matrix
plot_confusion_matrix(
    resnet_results['y_true'],
    resnet_results['y_pred'],
    'ResNet50'
)
plt.show()

# Print classification report
print_classification_report(
    resnet_results['y_true'],
    resnet_results['y_pred'],
    'ResNet50'
)

### 4.2 EfficientNetB0

In [ ]:
print("="*70)
print("Training EfficientNetB0")
print("="*70)

# Build model
effnet_model = build_transfer_learning_model('efficientnetb0', img_size=IMG_SIZE, num_classes=3)

# Train model
effnet_history = effnet_model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS,
    callbacks=get_callbacks('efficientnetb0', patience=15),
    verbose=1
)

# Plot training history
plot_training_history(effnet_history, 'EfficientNetB0')
plt.show()

print("\n✓ EfficientNetB0 training completed!")

In [ ]:
# Evaluate EfficientNetB0
print("Evaluating EfficientNetB0 on validation set...")
effnet_results = evaluate_model(effnet_model, val_generator)

print(f"\nEfficientNetB0 Results:")
print(f"  Accuracy:  {effnet_results['accuracy']:.4f}")
print(f"  Precision: {effnet_results['precision']:.4f}")
print(f"  Recall:    {effnet_results['recall']:.4f}")
print(f"  F1-Score:  {effnet_results['f1_score']:.4f}")

# Save results
save_model_results('efficientnetb0', effnet_results, effnet_history)

# Plot confusion matrix
plot_confusion_matrix(
    effnet_results['y_true'],
    effnet_results['y_pred'],
    'EfficientNetB0'
)
plt.show()

# Print classification report
print_classification_report(
    effnet_results['y_true'],
    effnet_results['y_pred'],
    'EfficientNetB0'
)

### 4.3 MobileNetV2

In [ ]:
print("="*70)
print("Training MobileNetV2")
print("="*70)

# Build model
mobilenet_model = build_transfer_learning_model('mobilenetv2', img_size=IMG_SIZE, num_classes=3)

# Train model
mobilenet_history = mobilenet_model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS,
    callbacks=get_callbacks('mobilenetv2', patience=15),
    verbose=1
)

# Plot training history
plot_training_history(mobilenet_history, 'MobileNetV2')
plt.show()

print("\n✓ MobileNetV2 training completed!")

In [ ]:
# Evaluate MobileNetV2
print("Evaluating MobileNetV2 on validation set...")
mobilenet_results = evaluate_model(mobilenet_model, val_generator)

print(f"\nMobileNetV2 Results:")
print(f"  Accuracy:  {mobilenet_results['accuracy']:.4f}")
print(f"  Precision: {mobilenet_results['precision']:.4f}")
print(f"  Recall:    {mobilenet_results['recall']:.4f}")
print(f"  F1-Score:  {mobilenet_results['f1_score']:.4f}")

# Save results
save_model_results('mobilenetv2', mobilenet_results, mobilenet_history)

# Plot confusion matrix
plot_confusion_matrix(
    mobilenet_results['y_true'],
    mobilenet_results['y_pred'],
    'MobileNetV2'
)
plt.show()

# Print classification report
print_classification_report(
    mobilenet_results['y_true'],
    mobilenet_results['y_pred'],
    'MobileNetV2'
)

## 5. Train YOLOv8 Classification Model

YOLOv8 also supports classification tasks and can be a strong competitor.

In [ ]:
# Prepare data in YOLO format
print("Preparing dataset for YOLO...")
yolo_data_dir = PROJECT_ROOT / 'Data' / 'yolo_damage_dataset'
yaml_path = load_dataset_for_yolo(yolo_data_dir)

print(f"\n✓ YOLO dataset prepared at: {yolo_data_dir}")
print(f"✓ Dataset config: {yaml_path}")

In [ ]:
print("="*70)
print("Training YOLOv8n Classification Model")
print("="*70)

# Train YOLOv8 (using nano model for faster training)
yolo_model, yolo_train_results = train_yolo_classifier(
    data_yaml_path=yaml_path,
    epochs=YOLO_EPOCHS,
    img_size=YOLO_IMG_SIZE,
    model_size='n',  # 'n' for nano, 's' for small, 'm' for medium
    project_name='yolov8n_damage',
)

print("\n✓ YOLOv8 training completed!")

In [ ]:
# Evaluate YOLOv8
print("Evaluating YOLOv8 on validation set...")
yolo_results = predict_yolo_batch(yolo_model, VAL_DIR)

print(f"\nYOLOv8 Results:")
print(f"  Accuracy:  {yolo_results['accuracy']:.4f}")
print(f"  Precision: {yolo_results['precision']:.4f}")
print(f"  Recall:    {yolo_results['recall']:.4f}")
print(f"  F1-Score:  {yolo_results['f1_score']:.4f}")

# Save results
save_yolo_results(yolo_results, 'yolov8n')

# Plot confusion matrix
plot_confusion_matrix(
    yolo_results['y_true'],
    yolo_results['y_pred'],
    'YOLOv8n'
)
plt.show()

# Print classification report
print_classification_report(
    yolo_results['y_true'],
    yolo_results['y_pred'],
    'YOLOv8n'
)

## 6. Summary of Results

In [ ]:
# Compile all results
all_results = {
    'ResNet50': resnet_results,
    'EfficientNetB0': effnet_results,
    'MobileNetV2': mobilenet_results,
    'YOLOv8n': yolo_results
}

# Create comparison DataFrame
comparison_data = {
    'Model': [],
    'Accuracy': [],
    'Precision': [],
    'Recall': [],
    'F1-Score': []
}

for model_name, results in all_results.items():
    comparison_data['Model'].append(model_name)
    comparison_data['Accuracy'].append(results['accuracy'])
    comparison_data['Precision'].append(results['precision'])
    comparison_data['Recall'].append(results['recall'])
    comparison_data['F1-Score'].append(results['f1_score'])

comparison_df = pd.DataFrame(comparison_data)
comparison_df = comparison_df.sort_values('Accuracy', ascending=False)

print("\n" + "="*80)
print("MODEL COMPARISON - VALIDATION SET PERFORMANCE")
print("="*80)
print(comparison_df.to_string(index=False))
print("="*80)

# Save comparison
comparison_df.to_csv(RESULTS_DIR / 'model_comparison.csv', index=False)
print(f"\n✓ Comparison saved to: {RESULTS_DIR / 'model_comparison.csv'}")

In [ ]:
# Visualize model comparison
comparison_dict = {
    model: {
        'accuracy': results['accuracy'],
        'precision': results['precision'],
        'recall': results['recall'],
        'f1_score': results['f1_score']
    }
    for model, results in all_results.items()
}

compare_models(comparison_dict)
plt.show()

print("\n✓ All models trained and evaluated successfully!")
print(f"\nBest Model: {comparison_df.iloc[0]['Model']} with {comparison_df.iloc[0]['Accuracy']:.4f} accuracy")

## 7. Next Steps

1. Open `02_model_comparison.ipynb` for detailed model analysis and visualization
2. Open `03_predict.ipynb` to test the trained models on new images
3. Consider fine-tuning the best performing model for even better results